In [ ]:
pip install rasterio geopandas shapely fiona pykml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
import rasterio
from shapely.geometry import box
import geopandas as gpd

# Load raster
raster_path = "/content/composite.tif"
output_dir = "kml_tiles"
os.makedirs(output_dir, exist_ok=True)

with rasterio.open(raster_path) as src:
    width = src.width
    height = src.height
    transform = src.transform
    crs = src.crs
    bounds = src.bounds

    tile_cols = 25
    tile_rows = 20

    # Get coordinate bounds from raster
    minx, miny = bounds.left, bounds.bottom
    maxx, maxy = bounds.right, bounds.top

    # Tile size in coordinate units (meters)
    tile_width = (maxx - minx) / tile_cols
    tile_height = (maxy - miny) / tile_rows

    tile_index = 1

    for i in range(tile_cols):
        for j in range(tile_rows):
            x1 = minx + i * tile_width
            x2 = x1 + tile_width
            y1 = maxy - j * tile_height
            y2 = y1 - tile_height

            geom = box(x1, y2, x2, y1)

            # Create GeoDataFrame
            gdf = gpd.GeoDataFrame({"id": [tile_index]}, geometry=[geom], crs=crs)

            # Reproject to WGS84 for Google Earth
            gdf = gdf.to_crs("EPSG:4326")

            # Export as KML
            kml_path = os.path.join(output_dir, f"tile_{tile_index:03d}.kml")
            gdf.to_file(kml_path, driver="KML")

            tile_index += 1

print("✅ Done: 500 polygon KMLs saved in", output_dir)


✅ Done: 500 polygon KMLs saved in kml_tiles


In [ ]:
from google.colab import files
import shutil

# Zip the folder
shutil.make_archive("kml_tiles", 'zip', "kml_tiles")

# Download the zipped folder
files.download("kml_tiles.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>